In [ ]:
import sys
import os
import tensorflow.keras
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import preprocess_input
import pandas as pd
from sklearn.model_selection import train_test_split
import PIL.Image as Image
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)
np.set_printoptions(threshold=sys.maxsize)

from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
from keras.layers import Dense, Activation, Flatten
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

def createModel():
  counter = 0
  resnet50 = tf.keras.applications.resnet50
  conv_model = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
  for layer in conv_model.layers:
      counter += 1
      layer.trainable = False

  x = tf.keras.layers.GlobalAveragePooling2D()(conv_model.output)
  predictions = tf.keras.layers.Dense(5, activation='softmax')(x)
  full_model = tf.keras.models.Model(inputs=conv_model.input, outputs=predictions)

  return full_model

In [ ]:
  # CREATE NEW MODEL
  model = createModel()
  # COMPILE NEW MODEL
  model.compile(loss='categorical_crossentropy',
          optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
          metrics=['accuracy', tf.keras.metrics.Precision()])

94773248/94765736 [==============================] - 1s 0us/step


In [ ]:
from tensorflow import keras

model = keras.models.load_model("/content/drive/My Drive/BachelorProject/temp_models/model_4.h5")

In [ ]:
resnet = createModel()

In [ ]:
thisdict = {
  "Bedroom": 0,
  "Kitchen": 1,
  "Living room": 2,
  "Restaurant,Bar" : 3,
  "Sport fields" : 4
}

In [ ]:
from tensorflow.keras.preprocessing import image
imgfiles = glob('/content/drive/My Drive/BachelorProject/HeatImages/living.jpg')

x = np.expand_dims(image.img_to_array(image.load_img(imgfiles[0],target_size=(224, 224, 3))),axis = 0)
x = preprocess_input(x)

In [ ]:
model.predict(x)

array([[0., 0., 1., 0., 0.]], dtype=float32)

In [ ]:
resnet.summary()

In [ ]:
activation_layer = model.get_layer('conv5_block3_out')

In [ ]:
from tensorflow.keras.models import Model

model1 = Model(inputs = model.input , outputs = activation_layer.output)

In [ ]:
final = resnet.get_layer('dense_1')
w = final.get_weights()

In [ ]:
maps = model1.predict(x)
probs = resnet.predict(x)

In [ ]:
import scipy as sp

pred = int(np.argmax(probs[0]))
W = w[0][pred]
cam = maps.dot(w[0].T[pred])
cam = sp.ndimage.zoom(cam[0], (32,32), order=1)

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1);
plt.imshow(cam , alpha=0.8);
plt.imshow(x[0] , alpha=0.2);
plt.subplot(1,2,2);
plt.imshow(x[0]);